<a href="https://colab.research.google.com/github/Yan344933/PythonPractice/blob/ExcelToJson/Json_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2021-06-06 19:32

Can't run get KMB
Connection error 104

---

2021-06-06 14:39

Exported CTB

going to export NWFB

Bug: Same orig and Dest for inbound and outbound (18:31 Fixed in excel)

---


 2021-06-05 23:10

Need to re-run CTB
-> Add seq in stop sheet
Run CTB, NWFB

KMB not yet implemented


In [ ]:
import pandas as pd
import requests

data_CTB = requests.get('https://rt.data.gov.hk/v1/transport/citybus-nwfb/route/CTB/').json()['data']
data_NWFB = requests.get('https://rt.data.gov.hk/v1/transport/citybus-nwfb/route/NWFB/').json()['data']

In [ ]:
## Reference only

df = pd.DataFrame.from_dict(data_CTB.json()['data'])
df2 = pd.DataFrame.from_dict(data_NWFB.json()['data'])
df = df.append(df2, ignore_index=True)

In [ ]:
routeList = []
route_count = 1
StopList = []
stop_count = 1

In [ ]:
CTB_Base = 'https://rt.data.gov.hk/'
directions = ['inbound', 'outbound']

In [ ]:
print(data_CTB)

In [ ]:
def addRoute(co, route_data, direction):
  global route_count
  route_dict = {}
  if co != 'KMB':
    route_dict['id'] = route_count
    route_dict['route'] = route_data['route']
    route_dict['CTB'] = True if co =='CTB' else False
    route_dict['NWFB'] = True if co =='NWFB' else False
    route_dict['KMB'] = False
    route_dict['direction'] = direction

    if direction = 'inbound':
      route_dict['origin'] = route_data['dest_tc']
      route_dict['destination'] = route_data['orig_tc']
    else:
      route_dict['origin'] = route_data['orig_tc']
      route_dict['destination'] = route_data['dest_tc']

    route_dict['service_type'] = 0
  else:
    route_dict['id'] = route_count
    route_dict['route'] = route_data['route']
    route_dict['CTB'] = False
    route_dict['NWFB'] = False
    route_dict['KMB'] = True
    route_dict['origin'] = route_data['orig_tc']
    route_dict['destination'] = route_data['dest_tc']
    route_dict['direction'] = 'inbound' if route_data['bound'] == 'I' else 'outbound'
    route_dict['service_type'] = int(route_data['service_type'])

  print('-- Add Route ' + route_dict['route'] + ' ' + route_dict['direction'] + ' ' + str(route_count) + ' --')
  route_count += 1
  routeList.append(route_dict)

In [ ]:
def addStop(stop_data, route_id, seq):
  global stop_count
  stop_dict = {}
  stop_dict['id'] = stop_count
  stop_dict['stop'] = stop_data['stop']
  stop_dict['route'] = route_id
  stop_dict['name'] = stop_data['name_tc']
  stop_dict['lat'] = stop_data['lat']
  stop_dict['long'] = stop_data['long']
  stop_dict['sequence'] = seq
  print('-- Add Stop ' + stop_dict['name'] + ' ' + str(stop_count) + ' --')
  stop_count += 1
  StopList.append(stop_dict)

In [ ]:
#for CTB and NWFB only

for route in data_CTB:
  # check if cycle route
  for direction in directions:
    stops = requests.get(CTB_Base + 'v1/transport/citybus-nwfb/route-stop/CTB/'+route['route']+'/'+direction).json()['data']
    if stops != []:
      addRoute('CTB', route, direction)
    for stop in stops:
      stop_data = requests.get(CTB_Base + 'v1/transport/citybus-nwfb/stop/' + stop['stop']).json()['data']
      addStop(stop_data, route_count-1, stop['seq'])

write_excel('CTB')

In [ ]:
routeList = []
route_count = 1
StopList = []
stop_count = 1

for route in data_NWFB:
  # check if cycle route
  for direction in directions:
    stops = requests.get(CTB_Base + 'v1/transport/citybus-nwfb/route-stop/NWFB/'+route['route']+'/'+direction).json()['data']
    if stops != []:
      addRoute('NWFB', route, direction)
    for stop in stops:
      stop_data = requests.get(CTB_Base + 'v1/transport/citybus-nwfb/stop/' + stop['stop']).json()['data']
      addStop(stop_data, route_count-1, stop['seq'])

write_excel('NWFB')

In [ ]:
# write to excel
from openpyxl import load_workbook
def write_excel(sheet_name):
  path = './drive/MyDrive/exported_excel.xlsx'
  #book = load_workbook(path)
  #writer = pd.ExcelWriter(path, engine = 'openpyxl')

  route_df = pd.DataFrame.from_dict(routeList)
  stop_df = pd.DataFrame.from_dict(StopList)

  with pd.ExcelWriter(path,mode='a') as writer:
    route_df.to_excel(writer, sheet_name=(sheet_name + ' route'))
    stop_df.to_excel(writer, sheet_name=(sheet_name + ' stop'))
    writer.save()

  print(str(route_count) + ' route records have been written.')
  print(str(stop_count) + ' stop records have been written.')
  #writer.close()

## KMB

In [ ]:
KMB_base = 'https://data.etabus.gov.hk/'
data_KMB = requests.get(KMB_base + 'v1/transport/kmb/route/').json()['data']

In [ ]:
for route in data_KMB[route_count-3:]:
  addRoute('KMB', route, "")
  route_stops = requests.get(KMB_base + 'v1/transport/kmb/route-stop/' + route['route'] + '/'+ ('inbound' if route['bound'] == 'I' else 'outbound') +
                                 '/' + route['service_type']).json()['data']
  for stop in route_stops:
    stop_data = requests.get(KMB_base + 'v1/transport/kmb/stop/' + stop['stop']).json()['data']
    addStop(stop_data, route_count - 1, stop['seq'])

write_excel('KMB')

In [ ]:
path = './drive/MyDrive/exported_excel.xlsx'
  #book = load_workbook(path)
  #writer = pd.ExcelWriter(path, engine = 'openpyxl')

route_df = pd.DataFrame.from_dict(routeList)
stop_df = pd.DataFrame.from_dict(StopList)

with pd.ExcelWriter(path,mode='a') as writer:
  route_df.to_excel(writer, sheet_name='KMB_Route')
  stop_df.to_excel(writer, sheet_name='KMB_Stop')
  writer.save()

In [ ]:
print(route_count)
print(stop_count)
print(data_KMB[route_count-3])
print(str(len(data_KMB)))

979
21667
{'route': '290B', 'bound': 'O', 'service_type': '1', 'orig_en': 'TSEUNG KWAN O INDUSTRIAL ESTATE', 'orig_tc': '將軍澳工業邨', 'orig_sc': '将军澳工业邨', 'dest_en': 'TSUEN WAN WEST STATION', 'dest_tc': '荃灣西站', 'dest_sc': '荃湾西站'}
1404
